# Building a Language Model
***
# Table of Contents
1.  [Setup](#Setup)
2.  [Coding Decisions](#Coding-Decisions)
3.  [Evaluation](#Evaluation)
4.  [Conclusion](#Conclusion)
5.  [References](#References)

# Setup

For this assignment I wrote the python package LanguageModel, code documentation and explanation is
included as docstrings inside the code. I put my particular coding and design choices in an md cell with the heading
[Coding Decisions](#Coding-Decisions). I am using the Maltese [[1]](#References) corpus dataset for this assignment
and python version 3.7.

I have also included an html file generated by jupyter notebooks and I recommend viewing that instead of using the
jupyter server. Alternatively I used the Jetbrains Pycharm IDE which also renders the md components neatly.

In [1]:
# Import the LanguageModel package
import LanguageModel as lm

# Coding Decisions

In this section I go over some coding decisions and/or design and why I went with them.

## Corpus

From the little big data applications I have worked so far I know that most big data applications make use of the numpy
library, or indirectly through the pandas library. I could have used numpy and made a CorpusAsListOfNpArrays but since
Sentences where originally an object of their own this did not cross my mind.

Another consideration was to hash/encode the words and use matrix operations to get the counts and probabilities. I
attempted this but the process was becoming to complicated and with no significant time improvement.

At the end I found python list syntax very easy to understand and use and the speed, combined with dictionaries was
sufficient.

## NGramCounts

The counts object represent the frequency count given n and model. I decided to only ever store vanilla counts because
when I implemented different counting methods, especially to account for non-appearing tokens, was becoming messy and
slow. By implementing a GetCount function I was able to achieve full functionality with clean code.

## NGramModel

Unlike the with the frequency counts for the probability set I calculate vanilla and laplace smoothed probabilities
differently. However the various methods of getting the probability for each ngram is then handled by the LanguageModel.


## LanguageModel

With the main class of the package I implemented most of the requirements of the assignment. I think I explain the code
good enough, most of the time with reasoning in the code's docstring and comments.

# Evaluation

In this section I create a number of LanguageModels on different corpus and evaluate them in a standard manner.

## Methodology

* First I will split the chosen corpus in an 80/20 training/testing split.

* I create a unigram, bigram, trigram and linear interpolation NGramModel for the three model types; vanilla, laplace
and unk. This is only done for the train LanguageModel.

* I create a unigram, bigram, trigram and linear interpolation NGramCounts for the three model types; vanilla, laplace
and unk. This is done for both LanguageModels.

* Test the test LanguageModel in the trained LanguageModel.

* Calculate the Test perplexity.

* Generate a number of sentences.

## Test Corpus

This corpus was created to test out the features of the package to make sure everything works as it is supposed to.

In [2]:
# Import train_test_split from sklearn and tqdm
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm


def getTrainTest(root):
    dataset = lm.Corpus.CorpusAsListOfSentences(root=root, verbose=True)
    train, test = train_test_split(dataset, test_size=0.2, shuffle=False)
    _train_lm = lm.LanguageModel(corpus=train, verbose=True)
    _test_lm = lm.LanguageModel(corpus=test, verbose=True)
    print("Train Corpus Size: ", _train_lm.GetNGramModel(n=1).N)
    print("Test Corpus Size: ", _test_lm.GetNGramModel(n=1).N)
    return _train_lm, _test_lm

train_lm, test_lm = getTrainTest(root='Test Corpus/')

Reading Files:   0%|          | 0/1 [00:00<?, ?it/s]

Parsing XML:   0%|          | 0/1 [00:00<?, ?it/s]

Building Sentences:   0%|          | 0/1 [00:00<?, ?it/s]

Paragraph:   0%|          | 0/1 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/8 [00:00<?, ?it/s]

...

Counting x counts:   0%|          | 0/8 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating Probabilities:   0%|          | 0/22 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/2 [00:00<?, ?it/s]

Train Corpus Size:  96
Test Corpus Size:  24


In this step I successfully split the training and testing data. The train LM has 96 words, 16 of which are start and
end tokens and the test LM has 24 words, 4 of which are start and end tokens.

In [3]:
params =    {
                "n": [1,2,3],
                "model": ["vanilla", "laplace", "unk"]
            }

def fitPredictTrain():
    for n in tqdm(params["n"]):
        for model in params["model"]:
            train_lm.GetNGramModel(n=n, model=model)
            train_lm.GetNGramModel(n=n, model=model)
            test_lm.GetNGramCounts(n=n, model=model)
fitPredictTrain()

  0%|          | 0/3 [00:00<?, ?it/s]

In this step I successfully generate the required data for the next step.

In [4]:
from collections import OrderedDict
from operator import itemgetter


unigram = OrderedDict()
bigram = OrderedDict()
trigram = OrderedDict()
interpolation = OrderedDict()

perplexity = {}

def predictTest():
    for n in tqdm(params["n"]):
        for model in params["model"]:
            # frequency counts from the test lm
            testgrams = test_lm.GetNGramCounts(n=n,model=model)
            # predict these ngrams using the trained model
            probabilities = {}
            for gram in testgrams:
                probabilities[gram] = train_lm.GetProbability(input=gram, n=n, model=model)
            # set the test lm model to these predictions
            test_lm.SetNGramModel(probabilities=probabilities, n=n, model=model)

            # Sort the probabilities, these will be used for visualization
            sorted_tuples = sorted(probabilities.items(), key=itemgetter(1))
            # fill the appropriate ordered dict
            if n == 1:
                unigram[model] = {}
                for k, v in sorted_tuples:
                    unigram[model][k] = v
            elif n == 2:
                bigram[model] = {}
                for k, v in sorted_tuples:
                    bigram[model][k] = v
            else:
                trigram[model] = {}
                for k, v in sorted_tuples:
                    trigram[model][k] = v

            # get the perplexity of the tested model
            perplexity[tuple([n, model])] = test_lm.Perplexity(n=n, model=model)

            if n == 3:
                interpolations = {}
                # predict the ngrams using the trained model
                for gram in testgrams:
                    interpolations[gram] = train_lm.LinearInterpolation(trigram=gram, model=model)
                 # Sort the probabilities, these will be used for visualization
                sorted_tuples = sorted(interpolations.items(), key=itemgetter(1))
                # fill the appropriate ordered dict
                interpolation[model] = {}
                for k, v in sorted_tuples:
                    interpolation[model][k] = v
                # get the perplexity of the linear interpolation tested model
                perplexity[tuple(['interpolation', model])] = test_lm.Perplexity(n=n, model=model, linearInterpolation=True)

predictTest()

  0%|          | 0/3 [00:00<?, ?it/s]

Now that I have successfully tested the corpus using my language model, I will now show some ngram probabilities and the
model perplexities.

In [5]:
heading =   "\t|\tUnigram\t\t|\tBigram\t\t\t|\tTrigram\t\t\t\t\t|\tLinear  Interpolation"
line =  "************************************************************************************************************"\
        "***************************************"
def visualizeWords():
    # This is just some me having fun with strings and python nothing else
    data_template =     "Vanilla\t|\t{}:{:.5f}%\t|\t{}:{:.5f}%\t|\t{}:{:.5f}%\t\t|\t{}:{:.5f}%\n" \
                        "Laplace\t|\t{}:{:.5f}%\t|\t{}:{:.5f}%\t|\t{}:{:.5f}%\t\t|\t{}:{:.5f}%\n" \
                        "UNK\t|\t{}:{:.5f}%\t|\t{}:{:.5f}%\t|\t{}:{:.5f}%\t\t|\t{}:{:.5f}%"

    words = []
    for i in range(min(len(unigram["unk"]), 5)):
        i = -i
        words.append(list(unigram["unk"].keys())[i])
        print(heading)
        print(line)
        print(data_template.format(
                " ".join([x for x in list(unigram["vanilla"].keys())[i]]),          (unigram["vanilla"][list(unigram["vanilla"].keys())[i]]) * 100,
                " ".join([x for x in list(bigram["vanilla"].keys())[i]]),           (bigram["vanilla"][list(bigram["vanilla"].keys())[i]]) * 100,
                " ".join([x for x in list(trigram["vanilla"].keys())[i]]),          (trigram["vanilla"][list(trigram["vanilla"].keys())[i]]) * 100,
                " ".join([x for x in list(interpolation["vanilla"].keys())[i]]),    (interpolation["vanilla"][list(interpolation["vanilla"].keys())[i]]) * 100,
                " ".join([x for x in list(unigram["laplace"].keys())[i]]),          (unigram["laplace"][list(unigram["laplace"].keys())[i]]) * 100,
                " ".join([x for x in list(bigram["laplace"].keys())[i]]),           (bigram["laplace"][list(bigram["laplace"].keys())[i]]) * 100,
                " ".join([x for x in list(trigram["laplace"].keys())[i]]),          (trigram["laplace"][list(trigram["laplace"].keys())[i]]) * 100,
                " ".join([x for x in list(interpolation["laplace"].keys())[i]]),    (interpolation["laplace"][list(interpolation["laplace"].keys())[i]]) * 100,
                " ".join([x for x in list(unigram["unk"].keys())[i]]),              (unigram["unk"][list(unigram["unk"].keys())[i]]) * 100,
                " ".join([x for x in list(bigram["unk"].keys())[i]]),               (bigram["unk"][list(bigram["unk"].keys())[i]]) * 100,
                " ".join([x for x in list(trigram["unk"].keys())[i]]),              (trigram["unk"][list(trigram["unk"].keys())[i]]) * 100,
                " ".join([x for x in list(interpolation["unk"].keys())[i]]),        (interpolation["unk"][list(interpolation["unk"].keys())[i]]) * 100))
        print(line)
visualizeWords()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	80:0.00000%	|	<s> 80:0.00000%	|	<s> 80 81:0.00000%		|	<s> 80 81:0.00000%
Laplace	|	80:0.01487%	|	<s> 80:0.01487%	|	<s> 80 81:0.01487%		|	<s> 80 81:0.01487%
UNK	|	unk:66.94215%	|	unk unk:75.52438%	|	unk unk unk:73.14751%		|	unk unk unk:73.24003%
***************************************************************************************************************************************************


The table gives us a glimpse of how much the various trained models in the train LM accommodate for the test LM.

Since the vanilla models do not accommodate for unknown words, the probability for these unknown ngrams is always 0,
however with the other 2 models we get better probabilities, especially for the unk model, since most of the words now,
in both the test and train lms are the unk token.

The unk probabilities are not a 100% because while the test lm converts the <s> and </s> tokens into unk tokens as well,
the train lm does not because there are more than 2 sentences. I would consider this as a feature and not a bug since
it can be seen as the unk model not giving much weight to sentence structure when the corpus does not have a lot of \
sentences much how it does this too other words.

In [6]:
def visualizePerplexity():
    # Somewhat cleaner than the one above
    perplexity_template =       "Vanilla\t|\t{:.2f}\t\t|\t{:.2f}\t\t\t|\t{:.2f}\t\t\t|\t{:.2f}\n" \
                                "Laplace\t|\t{:.2f}\t\t|\t{:.2f}\t\t\t|\t{:.2f}\t\t\t|\t{:.2f}\n" \
                                    "UNK\t|\t{:.2f}\t\t|\t{:.2f}\t\t\t|\t{:.2f}\t\t\t|\t{:.2f}"

    print(heading)
    print(line)
    print(perplexity_template.format(   perplexity[tuple([1, "vanilla"])], perplexity[tuple([2, "vanilla"])], perplexity[tuple([3, "vanilla"])], perplexity[tuple(["interpolation", "vanilla"])],
                                        perplexity[tuple([1, "laplace"])], perplexity[tuple([2, "laplace"])], perplexity[tuple([3, "laplace"])], perplexity[tuple(["interpolation", "laplace"])],
                                        perplexity[tuple([1, "unk"])],     perplexity[tuple([2, "unk"])],     perplexity[tuple([3, "unk"])],     perplexity[tuple(["interpolation", "unk"])]))
    print(line)
visualizePerplexity()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	0.00		|	0.00			|	0.00			|	0.00
Laplace	|	6477966.12		|	10406806.00			|	2395408.07			|	188.58
UNK	|	1.03		|	1.02			|	1.03			|	1.03
***************************************************************************************************************************************************


Given the context and the shown probabilities above, the perplexity of the models make sense. With the Vanilla models,
practically not accommodating the test lm, the laplace given such a big perplexity due to the very small accommodation
and unk having a very good almost 1 perplexity, again since most tokens are converted into unk tokens.

Now that I have evaluated the model intrinsically via perplexity, I can do a small extrinsic evaluation by generating two
sentences from each model in the trained Language Model. One will be given no start, while another will be given a
sequence for it to continue.

In [7]:
def generateFromEmpty():
    for n in tqdm(params["n"]):
        for model in params["model"]:
            print("n: {}\nmodel: {}\n".format(n,model))
            generated = train_lm.GenerateSentence(n=n, model=model, verbose=True)
            for w in generated:
                print(w, end=' ')
            print(".\n")
generateFromEmpty()

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

23 51 79 30 60 52 69 4 6 .

n: 1
model: laplace

31 46 8 74 56 33 8 .

n: 1
model: unk

.

n: 2
model: vanilla

10 11 12 13 14 15 16 17 18 19 .

n: 2
model: laplace

40 41 42 43 44 45 46 47 48 49 .

n: 2
model: unk

.

n: 3
model: vanilla

10 11 12 13 14 15 16 17 18 19 .

n: 3
model: laplace

10 11 12 13 14 15 16 17 18 19 .

n: 3
model: unk

.




The sentence generation output make sense for these reasons:

* Generating a sentence out of unk/<s>/</s> tokens is impossible.
* The bigram and trigram generations where able to complete the _0-_9 count.

In [8]:
def generateFrom(start):
    for n in tqdm(params["n"]):
        sentence = start[:-1]
        for model in params["model"]:
            generated = train_lm.GenerateSentence(start=start[-1], n=n, model=model, verbose=True)
            print("n: {}\nmodel: {}\n".format(n,model))
            given_and_generated = sentence + generated
            for w in given_and_generated:
                print(w, end=' ')
            print(".\n")
            
start = ['20', '21', '22']
generateFrom(start=start)

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

20 21 22 21 63 16 9 49 66 40 69 25 0 55 18 69 28 1 15 51 .

n: 1
model: laplace

20 21 22 6 42 8 4 41 75 57 50 3 35 56 23 24 .

n: 1
model: unk

20 21 22 .

n: 2
model: vanilla

20 21 22 23 24 25 26 27 28 29 .

n: 2
model: laplace

20 21 22 23 24 25 26 27 28 29 .

n: 2
model: unk

20 21 22 .

n: 3
model: vanilla

20 21 22 23 24 25 26 27 28 29 .

n: 3
model: laplace

20 21 22 23 24 25 26 27 28 29 .

n: 3
model: unk

20 21 22 .



The sentence generation output make sense for these reasons:

* Generating a sentence out of unk/<s>/</s> tokens is impossible.
* The bigram and trigram generations where able to complete the 20-29 count.

Now I will repeat the above steps for the other corpus

## Sports Corpus

This corpus is a subset of the larger complete Maltese corpus.

In [9]:
train_lm, test_lm = getTrainTest(root='Sports/')
fitPredictTrain()
unigram = OrderedDict()
bigram = OrderedDict()
trigram = OrderedDict()
interpolation = OrderedDict()
perplexity = {}
predictTest()

Reading Files:   0%|          | 0/2 [00:00<?, ?it/s]

Parsing XML:   0%|          | 0/2 [00:00<?, ?it/s]

Building Sentences:   0%|          | 0/2 [00:00<?, ?it/s]

Paragraph:   0%|          | 0/6 [00:00<?, ?it/s]

Paragraph:   0%|          | 0/4 [00:00<?, ?it/s]

...

Calculating Probabilities:   0%|          | 0/34 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/3 [00:00<?, ?it/s]

Train Corpus Size:  192
Test Corpus Size:  40


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Visualize some word tokens.

In [10]:
visualizeWords()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	din:0.00000%	|	<s> din:0.00000%	|	<s> din l:0.00000%		|	din l aħbar:0.00000%
Laplace	|	delre:0.00419%	|	<s> din:0.00731%	|	<s> din l:0.00731%		|	l istess delre:0.00699%
UNK	|	<s>:32.73160%	|	<s> unk:29.24005%	|	<s> unk l:21.22207%		|	<s> unk l:24.77842%
***************************************************************************************************************************************************
	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	</s>:4.68750%	|	2014 </s>:0.00000%	|	© 2014 </s>:0.00000%		|	© 2014 </s>:0.46875%
Laplace	|	</s>:0.10473%	|	2014 </s>:0.00731%	|	© 2014 </s>:0.00731%		|	© 2014 

Visualize the perplexity.

In [11]:
visualizePerplexity()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	0.00		|	0.00			|	0.00			|	0.00
Laplace	|	6489945.47		|	44904274.95			|	10760487.07			|	438.93
UNK	|	1.25		|	1.36			|	1.72			|	1.62
***************************************************************************************************************************************************


Generate sentences from the start token.

In [12]:
generateFromEmpty()

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

.

n: 1
model: laplace

fuq scicluna li sport aktar ieħor l eċċ opel swift swift tal .

n: 1
model: unk

asmk l li fil ta li fuq l asmk fuq u muturi tal li karozzi u li il u .

n: 2
model: vanilla

fl aħħarnett il heats kollha .

n: 2
model: laplace

fl ewwel attvitá ħadu sehem 27 ta sewwieqa ġodda .

n: 2
model: unk

fuq rebaħ il ħadd tal ta tal il ħadd li karozzi u l muturi .

n: 3
model: vanilla

fil finali ta klassi a rebaħ christian galea fuq il ħadd 27 ta ottubru 2013 l maltemp li fil finali ta klassi b rebaħ il heats .

n: 3
model: laplace

intant nhar il ħadd mis sewwieqa ġodda .

n: 3
model: unk

l li fil ħadd tal asmk il li fil li fuq il ħadd ta l li l asmk muturi u l muturi u karozzi u l .



Generate sentences from dr george abela.

In [13]:
start = ['dr', 'george', 'abela']
generateFrom(start=start)

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

dr george abela mario kien il filgħodu attività jkun filgħodu ta l sewwieqa b fiat l għalkemm kmieni klassi corsa rebaħ tilqà lija il ħakkem delre .

n: 1
model: laplace

dr george abela suzuki nicki kollha delre bil nhar waqt fil fl dan ta b .

n: 1
model: unk

dr george abela il l tal tal li .

n: 2
model: vanilla

dr george abela li ħakkem il ħadd 27 karozza li fil ġurnata għalkemm ħadd 6 ta klassi a rebaħ christian galea fuq suzuki swift ġabru l maltemp .

n: 2
model: laplace

dr george abela li fil finali ta sewwieqa ġodda .

n: 2
model: unk

dr george abela .

n: 3
model: vanilla

dr george abela li se jkun qed jattendi waqt attività oħra mill asmk frans deguara ppreżenta t trofej lir rebbieħa kollha tal ġurnata għalkemm ħadd mis sewwieqa .

n: 3
model: laplace

dr george abela li se jkun qed jattendi waqt l attività oħra mill asmk bil karozzi u karozzi bdiet staġun ieħor tal attivijiet bil karozzi u l .

n: 3
model: unk

dr george abela .



## Religion Corpus

This corpus is a subset of the larger complete Maltese corpus.

In [14]:
train_lm, test_lm = getTrainTest(root='Religion/')
fitPredictTrain()
unigram = OrderedDict()
bigram = OrderedDict()
trigram = OrderedDict()
interpolation = OrderedDict()
perplexity = {}
predictTest()

Reading Files:   0%|          | 0/2 [00:00<?, ?it/s]

Parsing XML:   0%|          | 0/2 [00:00<?, ?it/s]

Building Sentences:   0%|          | 0/2 [00:00<?, ?it/s]

Paragraph:   0%|          | 0/9 [00:00<?, ?it/s]

Paragraph:   0%|          | 0/34 [00:00<?, ?it/s]

...

Calculating Probabilities:   0%|          | 0/208 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/21 [00:00<?, ?it/s]

Train Corpus Size:  1407
Test Corpus Size:  388


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Visualize some word tokens.

In [15]:
visualizeWords()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	jgħannulu:0.00000%	|	ħa jgħannulu:0.00000%	|	<s> ħa jgħannulu:0.00000%		|	<s> ħa jgħannulu:0.00000%
Laplace	|	ħa:0.00010%	|	ħa jgħannulu:0.00026%	|	<s> ħa jgħannulu:0.00026%		|	min dahal ghax:0.00024%
UNK	|	<s>:15.68370%	|	<s> unk:14.50730%	|	<s> unk unk:9.66034%		|	<s> unk unk:11.71676%
***************************************************************************************************************************************************
	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	</s>:5.75693%	|	belongs to:100.00000%	|	maltese church </s>:100.00000%		|	the church the:6.35537%
Laplace	|	</s>:0.16365%	|	t

Visualize the perplexity.

In [16]:
visualizePerplexity()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	0.00		|	0.00			|	0.00			|	0.00
Laplace	|	710344.98		|	2785626405.97			|	3627868188.75			|	10961.74
UNK	|	1.31		|	2.52			|	6.55			|	5.51
***************************************************************************************************************************************************


Generate sentences from the start token.

In [17]:
generateFromEmpty()

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

li kien church 12 jew le mara .

n: 1
model: laplace

xagħrha kollha but rajh it i forma decided becoming numru fil quddiesa tal in owned belongs qatlitha .

n: 1
model: unk

john quddiesa to .

n: 2
model: vanilla

was midnight mass not belong to combine the birth of labour dominance .

n: 2
model: laplace

imma anqas għaraftek … weġibha alla jew ghax qieghed jezisti xi nghidu ghal kant taghna ma wegħdtnix li saħansitra kien ghad dar ta nofs il .

n: 2
model: unk

is it is in there alla at st john church and imma minn mill breakfast fil tal president .

n: 3
model: vanilla

disgrace .

n: 3
model: laplace

meta ghamilt kuncerti jien nahseb li kien jinkludi wkoll early breakfast fil kon katidral ta san gwann .

n: 3
model: unk

i kienet in there the mass at st john s co cathedral does not to sabet ruħha mill dar tal flus meta ma għall malta .



Generate sentences from dar missieri ghamiltuha.

In [18]:
start = ['dar', 'missieri', 'ghamiltuha']
generateFrom(start=start)

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

dar missieri ghamiltuha riedet b john kulur is .

n: 1
model: laplace

dar missieri ghamiltuha market lill attakk xorta għamlet not with coins turmoil quddiesa opportunity sold lill became oppose tgergir kon flus belt bl put alla l mercenari .

n: 1
model: unk

dar missieri ghamiltuha .

n: 2
model: vanilla

dar missieri ghamiltuha bejta tal kon katidral ma jhalluk qatt tinkludi xi nghidu ghal kant taghna ma nistax nifhem din il mulej .

n: 2
model: laplace

dar missieri ghamiltuha bejta tal flus .

n: 2
model: unk

dar missieri ghamiltuha .

n: 3
model: vanilla

dar missieri ghamiltuha bejta tal president il belt valletta becoming like a market place during this term of labour dominance .

n: 3
model: laplace

dar missieri ghamiltuha bejta tal qalb u tmint ijiem x tgħix .

n: 3
model: unk

dar missieri ghamiltuha .



## Maltese Corpus

The complete Maltese corpus.

In [19]:
train_lm, test_lm = getTrainTest(root='Maltese/')
fitPredictTrain()
unigram = OrderedDict()
bigram = OrderedDict()
trigram = OrderedDict()
interpolation = OrderedDict()
perplexity = {}
predictTest()

Reading Files:   0%|          | 0/28 [00:00<?, ?it/s]

Parsing XML:   0%|          | 0/28 [00:00<?, ?it/s]

Building Sentences:   0%|          | 0/28 [00:00<?, ?it/s]

Paragraph:   0%|          | 0/10 [00:00<?, ?it/s]

Paragraph:   0%|          | 0/39 [00:00<?, ?it/s]

...

Calculating Probabilities:   0%|          | 0/3312 [00:00<?, ?it/s]

Counting x counts:   0%|          | 0/809 [00:00<?, ?it/s]

Train Corpus Size:  73874
Test Corpus Size:  15796


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Visualize some word tokens.

In [20]:
visualizeWords()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	email:0.00000%	|	ħaġa interessanti:0.00000%	|	<s> xi ħaġa:0.00000%		|	oppożizzjoni bagħat email:0.00000%
Laplace	|	bagħat:0.00000%	|	ħaġa interessanti:0.00000%	|	<s> xi ħaġa:0.00000%		|	l oppożizzjoni bagħat:0.00000%
UNK	|	<s>:0.99339%	|	<s> xi:0.67404%	|	<s> xi ħaġa:0.19513%		|	<s> xi ħaġa:0.41863%
***************************************************************************************************************************************************
	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	</s>:4.37772%	|	within the:100.00000%	|	abela li se:100.00000%		|	of the 3:61.31057%
Laplace	|	</s>:0.15216%	|	ta

Visualize the perplexity.

In [21]:
visualizePerplexity()

	|	Unigram		|	Bigram			|	Trigram					|	Linear  Interpolation
***************************************************************************************************************************************************
Vanilla	|	0.00		|	0.00			|	0.00			|	0.00
Laplace	|	2168.72		|	1224058620.70			|	171419243566.46			|	110699.08
UNK	|	1.63		|	27.79			|	1199.05			|	404.49
***************************************************************************************************************************************************


Generate sentences from the start token.

In [22]:
generateFromEmpty()

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

ukoll u mingħajr u gvern turi president tal għal ta jidħol sabiex liġi li akkuża aqwa jitkellem qiegħed żona influwenza identita data jservi nimplimentaw m .

n: 1
model: laplace

inutli sinjali mepa tan terzjarja u għadhom snin what fiż ġlieda persuna investiment istess l servizz u tiegħi enerġija li .

n: 1
model: unk

u unjoni art .

n: 2
model: vanilla

onor membri aye dawk li wieħed seta jagħżel li l bażi li jivvintaw affarijiet oħrajn li tixraq lil min hija tmien snin dejjem jiftaħru li .

n: 2
model: laplace

irridu noħolqu ambjent aħjar profitti akbar porzjon ta l qagħda finanzjarja biex jemenda l għaqda fil komunità minn hekk l użu miż żewġ direzzjonijiet l .

n: 2
model: unk

nifhem għaliex għadhom jidhru bħala mepa ħalli nagħtih l identity card min magħha fuq kwestjonijiet kif ukoll jekk by għal din issir .

n: 3
model: vanilla

bћalissa gћaddej xogħol importanti u ambizzjuż mal gżira tibqa tirreġistra progress partikolarment il kriżi tal prezz tiegħu .


Generate sentences from dar missieri ghamiltuha.

In [23]:
start = ['dar', 'missieri', 'ghamiltuha']
generateFrom(start=start)

  0%|          | 0/3 [00:00<?, ?it/s]

n: 1
model: vanilla

dar missieri ghamiltuha l reġistrazzjoni meta kwarti tal u u hemm dovuta l kontra minn l nkomplu tal istatus t tagħna meta ta dan għall .

n: 1
model: laplace

dar missieri ghamiltuha produzzjoni ġiet imma .

n: 1
model: unk

dar missieri ghamiltuha direzzjoni wieħed .

n: 2
model: vanilla

dar missieri ghamiltuha .

n: 2
model: laplace

dar missieri ghamiltuha .

n: 2
model: unk

dar missieri ghamiltuha .

n: 3
model: vanilla

dar missieri ghamiltuha .

n: 3
model: laplace

dar missieri ghamiltuha .

n: 3
model: unk

dar missieri ghamiltuha .



By using the same start sequence I wanted to see whether the complete LM would generate a similar sentence to that of
the Religion lm. However this did not happen.

# Conclusion

From the above tests and results I can conclude that for a NGram Language Model to be truly effective, the corpus given
to it must be very accommodating. Moreover I saw the effect that the unk and laplace models had and how easily the
vanilla model can be broken. Since only 1 0-probability would result in a 0 perplexity.

# References

[1] Gatt, A., & Čéplö, S., Digital corpora and other electronic resources for Maltese. In A. Hardie, & R. Love (Eds.), Corpus Linguistics, 2013, pp. 96-97

[2] G. Pibiri and R. Venturini, "Handling Massive N -Gram Datasets Efficiently", ACM Transactions on Information Systems, vol. 37, no. 2, pp. 1-41, 2019. Available: 10.1145/3302913 [Accessed 8 April 2021].
https://towardsdatascience.com/perplexity-in-language-models-87a196019a94